In [14]:
import os
from pyprojroot import here
os.environ["HF_HOME"] = str(here("cache/huggingface"))
import lightning as L
from importlib import reload
import sys
sys.path.append(str(here()))
from models import model_definitions as MD
import torch
import pandas as pd
reload(MD)
torch.set_float32_matmul_precision('medium')
data_module_3 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_3.parquet")))
data_module_7 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_7.parquet")))
data_module_28 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_28.parquet")))
metrics = MD.MetricsCallback()

def make_trainer():
  return L.Trainer(
    max_epochs=15,
    max_time={"minutes": 10},
    callbacks=[
      # metrics,
    ],
    deterministic=True,
    default_root_dir=here("cache/lightning"),
    enable_checkpointing=True,
    fast_dev_run=True,
    accelerator="auto", 
  )

trainer = make_trainer()
trainer

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


In [ ]:
reload(MD)
distil_3 = MD.DistilBertFinetune(n_emotions=3)
if False:    
  trainer.fit(
    distil_3,
    datamodule=data_module_3,
  )
else:
  distil_3 = MD.DistilBertFinetune.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_3_14epchs.ckpt"),
    n_emotions=3,
  )
out = trainer.test(
  distil_3,
  datamodule=data_module_3,
)
del distil_3
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\Plancha\emotion-recognition\.pixi\envs\gpu\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_work

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
    test_f1_interest        0.5431039929389954
      test_f1_stand         0.5898196697235107
        test_loss           0.4733329713344574
        test_rmse           0.25767073035240173
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.4733329713344574,
  'test_f1_stand': 0.5898196697235107,
  'test_f1_interest': 0.5431039929389954,
  'test_rmse': 0.25767073035240173}]

In [ ]:
reload(MD)
distil_7 = MD.DistilBertFinetune(n_emotions=7)
if False:    
  trainer.fit(
    distil_7,
    data_module=data_module_7,
  )
else:
  distil_7 = MD.DistilBertFinetune.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_7_14epchs.ckpt"),
    n_emotions=7,
  )
out = trainer.test(
  distil_7,
  datamodule=data_module_7
)
del distil_7
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
    test_f1_interest        0.2703874707221985
      test_f1_stand         0.22714023292064667
        test_loss           0.28195345401763916
        test_rmse           0.18735003471374512
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.28195345401763916,
  'test_f1_stand': 0.22714023292064667,
  'test_f1_interest': 0.2703874707221985,
  'test_rmse': 0.18735003471374512}]

In [6]:
reload(MD)
distil_28 = MD.DistilBertFinetune(n_emotions=28)
if False:    
  trainer.fit(
    distil_28,
    datamodule=data_module_28,
  )
else:
  distil_28 = MD.DistilBertFinetune.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_28_11epchs.ckpt"),
    n_emotions=28,
  )
out = trainer.test(
  distil_28,
  datamodule=data_module_28,
)
del distil_28
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
    test_f1_interest        0.08144357800483704
      test_f1_stand         0.0910865068435669
        test_loss           0.11788760870695114
        test_rmse           0.11157042533159256
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.11788760870695114,
  'test_f1_stand': 0.0910865068435669,
  'test_f1_interest': 0.08144357800483704,
  'test_rmse': 0.11157042533159256}]

In [ ]:
reload(MD) # DistilBertFinetuneOnDCG
distil_dcg_3 = MD.DistilBertFinetuneOnDCG(n_emotions=3)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_dcg_3,
    datamodule=data_module_3,
  )
else:
  distil_dcg_3 = MD.DistilBertFinetuneOnDCG.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_dcg_3.ckpt"),
    n_emotions=3,
  ) 
out = trainer.test(
  distil_dcg_3,
  datamodule=data_module_3,
)
del distil_dcg_3
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                                | Params | Mode 
------------------------------------------------------------------------------
0 | model         | DistilBertForSequenceClassification | 67.0 M | train
1 | sigmoid       | Sigmoid                             | 0      | train
2 | sig_loss      | BCEWithLogitsLoss                   | 0      | train
3 | nDGC          | NDCG                                | 0      | train
4 | expected_nDGC | SoftRankExpectedNDCG                | 0      | train
5 | f1_stand      | MultilabelF1Score                   | 0      | train
6 | f1_interes

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\Plancha\emotion-recognition\.pixi\envs\gpu\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.6764426231384277
    test_expectedNDCG       0.7026176452636719
    test_f1_interest        0.13991768658161163
      test_f1_stand         0.2442244291305542
        test_nDGC           0.7973607182502747
        test_rmse           0.40463322401046753
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
reload(MD)
distil_dcg_7 = MD.DistilBertFinetuneOnDCG(n_emotions=7)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_dcg_7,
    datamodule=data_module_7,
  )
else:
  distil_dcg_7 = MD.DistilBertFinetuneOnDCG.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_dcg_7.ckpt"),
    n_emotions=7,
  )
  
out = trainer.test(
  distil_dcg_7,
  datamodule=data_module_7,
)
del distil_dcg_7
out

In [ ]:
reload(MD)
distil_dcg_28 = MD.DistilBertFinetuneOnDCG(n_emotions=28)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_dcg_28,
    datamodule=data_module_28,
  )
else:
  distil_dcg_28 = MD.DistilBertFinetuneOnDCG.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_dcg_28.ckpt"),
    n_emotions=28,
  )
out = trainer.test(
  distil_dcg_28,
  datamodule=data_module_28,
)
del distil_dcg_28
out